In [333]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [334]:
df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
combine = [df, test_df]

In [335]:
null_cols = combine[0].isnull().sum().where(combine[0].isnull().sum() > 0)[combine[0].isnull().sum().where(combine[0].isnull().sum() > 0) > 0] # there has to be a better way but im js like that 
null_cols

LotFrontage      259.0
Alley           1369.0
MasVnrType       872.0
MasVnrArea         8.0
BsmtQual          37.0
BsmtCond          37.0
BsmtExposure      38.0
BsmtFinType1      37.0
BsmtFinType2      38.0
Electrical         1.0
FireplaceQu      690.0
GarageType        81.0
GarageYrBlt       81.0
GarageFinish      81.0
GarageQual        81.0
GarageCond        81.0
PoolQC          1453.0
Fence           1179.0
MiscFeature     1406.0
dtype: float64

In [336]:
# data cleaning
null_cols_80 = null_cols[null_cols > len(combine[0].index) * 0.8].index.values # re check how many %
null_cols_80 = [n.strip() for n in null_cols_80]
combine[0] = combine[0].drop(null_cols_80, axis=1)
combine[1] = combine[1].drop(null_cols_80, axis=1)
print(f'dropped columns: ', null_cols_80)

for i in range(len(combine)):
    combine[i]['LotFrontage'] = combine[i]['LotFrontage'].fillna(combine[0]['LotFrontage'].median())
    combine[i]['MasVnrArea'] = combine[i]['MasVnrArea'].fillna(combine[0]['MasVnrArea'].median())
    combine[i]['BsmtFinSF1'] = combine[i]['BsmtFinSF1'].fillna(combine[0]['BsmtFinSF1'].median())
    combine[i]['BsmtFinSF2'] = combine[i]['BsmtFinSF2'].fillna(combine[0]['BsmtFinSF2'].median())
    combine[i]['GarageYrBlt'] = combine[i]['GarageYrBlt'].fillna(combine[0]['GarageYrBlt'].median())
    combine[i]['Electrical'] = combine[i]['Electrical'].fillna(combine[0]['Electrical'].mode()[0])

dropped columns:  ['Alley', 'PoolQC', 'Fence', 'MiscFeature']


In [337]:
combine[0].isnull().sum().where(combine[0].isnull().sum() > 0)[combine[0].isnull().sum().where(combine[0].isnull().sum() > 0) > 0] # there has to be a better way but im js like that 

MasVnrType      872.0
BsmtQual         37.0
BsmtCond         37.0
BsmtExposure     38.0
BsmtFinType1     37.0
BsmtFinType2     38.0
FireplaceQu     690.0
GarageType       81.0
GarageFinish     81.0
GarageQual       81.0
GarageCond       81.0
dtype: float64

In [338]:
categorical_cols = combine[0].select_dtypes(include=['object', 'str'])
numerical_cols = combine[0].select_dtypes(include=['int', 'float']).columns.drop(['Id'])
to_scale_numerical_cols = numerical_cols.drop(['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFullBath' ,'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars',  'MoSold', 'YrSold', 'SalePrice'])

In [339]:
MSSubClass_mapping = {
    20: 0,
    30: 1,
    40: 2,
    45: 3,
    50: 4,
    60: 5,
    70: 6,
    75: 7,
    80: 8,
    85: 9,
    90: 10,
    120: 11,
    150: 12,
    160: 13,
    180: 14,
    190: 15
}

Street_mapping = {
    "Grvl": 0,
    "Pave": 1
}

LandSlope_mapping = {
    "Sev": 0,
    "Mod": 1,
    "Gtl": 2
}

ExterQual_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

ExterCond_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

BsmtQual_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

BsmtCond_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

BsmtExposure_mapping = {
    "NA": 0,
    "No": 1,
    "Mn": 2,
    "Av": 3,
    "Gd": 4
}

BsmtFinType1_mapping = {
    "NA": 0,
    "Unf": 1,
    "LwQ": 2,
    "Rec": 3,
    "BLQ": 4,
    "ALQ": 5,
    "GLQ": 6
}

BsmtFinType2_mapping = BsmtFinType1_mapping.copy()

HeatingQC_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

CentralAir_mapping = {
    "N": 0,
    "Y": 1
}

KitchenQual_mapping = {
    "Po": 0,
    "Fa": 1,
    "TA": 2,
    "Gd": 3,
    "Ex": 4
}

FireplaceQu_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

GarageFinish_mapping = {
    "NA": 0,
    "Unf": 1,
    "RFn": 2,
    "Fin": 3
}

GarageQual_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

GarageCond_mapping = {
    "NA": 0,
    "Po": 1,
    "Fa": 2,
    "TA": 3,
    "Gd": 4,
    "Ex": 5
}

PavedDrive_mapping = {
    "N": 0,
    "P": 1,
    "Y": 2
}

mappings = {
    "MSSubClass": MSSubClass_mapping,
    "Street": Street_mapping,
    "LandSlope": LandSlope_mapping,
    "ExterQual": ExterQual_mapping,
    "ExterCond": ExterCond_mapping,
    "BsmtQual": BsmtQual_mapping,
    "BsmtCond": BsmtCond_mapping,
    "BsmtExposure": BsmtExposure_mapping,
    "BsmtFinType1": BsmtFinType1_mapping,
    "BsmtFinType2": BsmtFinType2_mapping,
    "HeatingQC": HeatingQC_mapping,
    "CentralAir": CentralAir_mapping,
    "KitchenQual": KitchenQual_mapping,
    "FireplaceQu": FireplaceQu_mapping,
    "GarageFinish": GarageFinish_mapping,
    "GarageQual": GarageQual_mapping,
    "GarageCond": GarageCond_mapping,
    "PavedDrive": PavedDrive_mapping,
}

print(combine[0].dtypes.value_counts())

for i in range(len(combine)):
    mapped_col = []
    for col, mapping in mappings.items():
        mapped_col.append(col)
        combine[i][col] = combine[i][col].map(mapping).fillna(0)

print(combine[0].dtypes.value_counts())
combine[0].isnull().sum().where(combine[0].isnull().sum() > 0)[combine[0].isnull().sum().where(combine[0].isnull().sum() > 0) > 0] # there has to be a better way but im js like that 

str        39
int64      35
float64     3
Name: count, dtype: int64
int64      43
str        22
float64    12
Name: count, dtype: int64


MasVnrType    872.0
GarageType     81.0
dtype: float64

In [340]:
# one hot 
# df['Electrical'] = df['Electrical'].astype('str')

not_mapped = [item for item in categorical_cols if item not in mapped_col]
if 'SalePrice' in not_mapped:
    print('bego')

# print(combine[0].dtypes.value_counts())
# print(combine[1].dtypes.value_counts())
print(len(combine[0].columns))
print(len(combine[1].columns))

one_hot = pd.concat(combine, axis=0)

one_hot = pd.get_dummies(one_hot, columns=not_mapped, dtype='int')

combine[0] = one_hot.iloc[:len(combine[0]), :].copy()
combine[1] = one_hot.iloc[len(combine[0]):, :].copy()

# print(combine[0].dtypes.value_counts())
# print(combine[1].dtypes.value_counts())
print(len(combine[0].columns))
print(len(combine[1].columns))

77
76
223
223


In [341]:
for dataset in combine:

    dataset['YearBuiltBand'] = pd.qcut(dataset['YearBuilt'], 4)

    dataset.loc[dataset['YearBuilt'] <= 1954, 'YearBuilt'] = 0
    dataset.loc[(dataset['YearBuilt'] > 1954) & (dataset['YearBuilt'] <= 1973), 'YearBuilt'] = 1
    dataset.loc[(dataset['YearBuilt'] > 1973) & (dataset['YearBuilt'] <= 2000), 'YearBuilt'] = 2
    dataset.loc[(dataset['YearBuilt'] > 2000) & (dataset['YearBuilt'] <= 2010), 'YearBuilt'] = 3
    dataset['YearBuilt'] = dataset['YearBuilt'].astype(int)

    dataset['YearRemodAddBand'] = pd.qcut(dataset['YearRemodAdd'], 4)

    dataset.loc[dataset['YearRemodAdd'] <= 1967, 'YearRemodAdd'] = 0
    dataset.loc[(dataset['YearRemodAdd'] > 1967) & (dataset['YearRemodAdd'] <= 1994), 'YearRemodAdd'] = 1
    dataset.loc[(dataset['YearRemodAdd'] > 1994) & (dataset['YearRemodAdd'] <= 2004), 'YearRemodAdd'] = 2
    dataset.loc[(dataset['YearRemodAdd'] > 2004) & (dataset['YearRemodAdd'] <= 2010), 'YearRemodAdd'] = 3
    dataset['YearRemodAdd'] = dataset['YearRemodAdd'].astype(int)

    dataset['GarageYrBltBand'] = pd.qcut(dataset['GarageYrBlt'], 4)

    dataset.loc[dataset['GarageYrBlt'] <= 1958, 'GarageYrBlt'] = 0
    dataset.loc[(dataset['GarageYrBlt'] > 1958) & (dataset['GarageYrBlt'] <= 1977), 'GarageYrBlt'] = 1
    dataset.loc[(dataset['GarageYrBlt'] > 1977) & (dataset['GarageYrBlt'] <= 2001), 'GarageYrBlt'] = 2
    dataset.loc[(dataset['GarageYrBlt'] > 2001) & (dataset['GarageYrBlt'] <= 2010), 'GarageYrBlt'] = 3
    dataset['GarageYrBlt'] = dataset['GarageYrBlt'].astype(int)

    dataset.drop(['YearBuiltBand', 'YearRemodAddBand', 'GarageYrBltBand'], axis=1, inplace=True)

In [342]:
# scale
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

for i in range(len(combine)):
    combine[i][to_scale_numerical_cols] = scaler.fit_transform(combine[i][to_scale_numerical_cols])

In [343]:
# split
X_train = combine[0].drop(['SalePrice', 'Id'], axis=1)
y_train = combine[0]['SalePrice']

print(X_train.info())
print(X_test.info())

id = combine[1]['Id']
X_test = combine[1].drop(['SalePrice', 'Id'], axis=1)
# y_test = combine[1]['SalePrice']

train_cols = X_train.columns.values.tolist()
test_cols = X_test.columns.values.tolist()

diff = set(train_cols) - set(test_cols)
print(diff)

X_train.drop(columns=diff, inplace=True)

# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

<class 'pandas.DataFrame'>
Index: 1460 entries, 0 to 1459
Columns: 221 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(32), int64(189)
memory usage: 2.5 MB
None
<class 'pandas.DataFrame'>
Index: 1459 entries, 0 to 1458
Columns: 221 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(32), int64(189)
memory usage: 2.5 MB
None
set()


In [344]:
#y_train.info()
y_train.info()

<class 'pandas.Series'>
Index: 1460 entries, 0 to 1459
Series name: SalePrice
Non-Null Count  Dtype  
--------------  -----  
1460 non-null   float64
dtypes: float64(1)
memory usage: 22.8 KB


In [345]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

preds = rf.predict(X_test)

submission = pd.DataFrame({
    "Id": id,
    "SalePrice": preds
})

submission.to_csv("submission.csv", index=False)


In [ ]:
print(X_train.info())
print(X_test.info())

train_cols = X_train.columns.values.tolist()
test_cols = X_test.columns.values.tolist()

diff = set(train_cols) - set(test_cols)
print(diff

<class 'pandas.DataFrame'>
Index: 1460 entries, 0 to 1459
Columns: 221 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(32), int64(189)
memory usage: 2.5 MB
None
<class 'pandas.DataFrame'>
Index: 1459 entries, 0 to 1458
Columns: 221 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(32), int64(189)
memory usage: 2.5 MB
None
set()
